# Cantidad de rutas de transporte público por cada calle

Este notebook toma de insumos ejes viales y hubdistance resultantes del "Procedimiento para trazar un mapa de las rutas de transporte público que pasan por cada calle" dentro de la carpeta de "Metodologías" en "Docs & Files" de "Innovación y Desarrollo" en el Basecamp del Observatorio de Ciudades. (26/04/2023)

Con esos insumos se genera un gdf con las calles y el número aproximado de rutas de transporte público que transitan en ellas.

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/usr/local/python/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


## Download data

In [3]:
#Cargar los ejes viales con el ID único CVEGEOSEG anteriormente generado
ejesviales = gpd.read_file('../../data/external/temporal_todocker/marcogeo_ejesviales_2km.gpkg')

#Cargar el Distance to nearest hub entre los puntos de las rutas y los puntos de las calles anteriormente generado
hubdistance = gpd.read_file('../../data/external/temporal_todocker/hubdistance_interpolatedpoints10m.gpkg')

In [4]:
print(ejesviales.shape)
ejesviales.head(1)

(19887, 14)


,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVEVIAL,CVESEG,NOMVIAL,TIPOVIAL,SENTIDO,TIPOSEN,AMBITO,CVEGEOSEG,rutas_transporte,geometry
0,140390001,14,039,0001,03975,00007,Patria,Avenida,Dos sentidos,2,Urbana,1403900010397500007,NaN,"MULTILINESTRING ((668243.519 2290955.957, 6682..."


In [5]:
print(hubdistance.shape)
hubdistance.head(2)

(136330, 10)


,fid,Tipo_de_se,Clasificac,Clasifica0,idruta,distance,angle,HubName,HubDist,geometry
0,11,Complementaria,Rutas complementarias y otros servicios,Operando,1,0.0,2.55326,1403900010366300013,5.275342,POINT (671655.450 2288709.068)
1,11,Complementaria,Rutas complementarias y otros servicios,Operando,1,10.0,2.55326,1403900010366300013,4.997463,POINT (671655.896 2288719.059)


In [6]:
#Filtrado de columnas para hubdistance
columns_tokeep = ['idruta','HubName','geometry']
hubdistance_f = hubdistance[columns_tokeep]
hubdistance_f = hubdistance_f.rename(columns={'HubName':'segmentocalle'})
hubdistance_f.head(1)

,idruta,segmentocalle,geometry
0,1,1403900010366300013,POINT (671655.450 2288709.068)


## Iteración (Método anterior, no permite ajustar el número de veces que se debe encontrar una ruta en un segmento para considerar que pasa por ahí y no solo es una intersección)

In [89]:
#hubdistance_f.head(2)

In [90]:
# ---------- Diccionario de segmentos que tienen rutas y las rutas que tienen
#dicc_segmentos_rutas = {}
#listavacia = []

# ---------- Crear un diccionario con los segmentos de calle y una lista de rutas por segmento
#for index,row in hubdistance_f.iterrows():    
#    segmento_calle = row['segmentocalle'] #Es un str   
#    ruta = row['idruta'] #Es un int
    
    # ---------- Si es la primera vez que encuentra este segmento, agregarlo con una lista vacía
#    if segmento_calle not in dicc_segmentos_rutas:
#        dicc_segmentos_rutas[segmento_calle] = listavacia.copy()

    # ---------- Para esta row en la que iteramos (segmento de calle y ruta), si es la primera vez que encuentra la ruta en este segmento, agregarla a esa lista:
#    if ruta not in dicc_segmentos_rutas[segmento_calle]:
#        dicc_segmentos_rutas[segmento_calle].append(ruta)

## Data processing 01 - Diccionario con segmentos y rutas que pasan por cada segmento

In [25]:
#Lista de rutas de transporte público
rutas = list(hubdistance_f.idruta.unique())

In [26]:
#---------- Insumos base de la iteración ----------
dicc_segmentos_rutas = {} #Guarda los segmentos de calle y las rutas que pasan por cada segmento de calle
listavacia = [] #Lista vacía usada cuando se encuentra un nuevo segmento de calle

#---------- Para decidir si es una intersección o una calle muy pequeña se usa mincount ----------

# mincount es el número mínimo de ocasiones en las que se debe encontrar un segmento de calle cerca de una ruta 
# para considerar que la ruta pasa por ahí y no solo que la encontró porque es una intersección

# ¿Cuántos puntos normalmente hay por segmento de calle? Con interpolated points rutas @10m y segmentos @10m  (Iteración actual): 
    # Cuando una ruta toma una calle de 60-70 metros, registra aprox 6 puntos
    # Cuando una ruta toma una calle de 60 metros, registra 4-6 puntos.
    # Cuando una ruta toma una calle de 50 metros o menos, registra 3 puntos o menos. (No muy común)
    # Cuando una ruta pasa por una intersección registra de 1 a 2 puntos de esa calle

# ¿Qué valor de mincount asignar? Depende de lo que se busca.
    # mincount de 9 genera muchos gaps en las rutas, pero no hay nada que limpiar en GIS (No toma en cuenta calles de intersecciones)
    # mincount de 5 genera pocos gaps en las rutas  y poco que limpiar en GIS (Toma algunas calles de intersecciones que hay que retirar)
    # mincount de 4 genera casi ningun gap en las rutas y mas o menos que limpiar en GIS (Toma varias intersecciones que hay que retirar)

mincount = 5 
    
#---------- Iterar ruta por ruta ----------
for ruta in rutas:
    gdf_ruta = hubdistance_f[hubdistance_f.idruta == ruta]

    #---------- Contar veces que esa ruta pasa por cada segmento de calle con groupby ----------
    segs_deruta = gdf_ruta.groupby('segmentocalle').count()
    #---------- Formato ----------
    columns_tokeep = ['idruta']
    segs_deruta = segs_deruta[columns_tokeep]
    segs_deruta = segs_deruta.reset_index()
    segs_deruta = segs_deruta.rename(columns={'idruta':'count'})

    #---------- Para el count de segmentos de esa ruta: ----------
    
    #Iterar por cada segmento y las veces que se encontró:
    for row_id,row_series in segs_deruta.iterrows():
        segmento_calle = row_series[0]
        count = row_series[1]
    
        #---------- Si la ruta pasa por ahí más veces de las que se requieren por mincount
        #---------- se confirma que la ruta pasa por ahí. Se agrega al diccionario ese segmento de calle y esa ruta.----------
        if count > (mincount-1): 
            #---------- Si no se había encontrado ese segmento de calle, meterlo al diccionario por primera vez con una lista vacia de rutas ----------
            if segmento_calle not in dicc_segmentos_rutas:
                dicc_segmentos_rutas[segmento_calle] = listavacia.copy()
            
            #---------- Si la ruta no está ya asignada a ese segmento, meterla a la lista ----------
            if ruta not in dicc_segmentos_rutas[segmento_calle]:
                dicc_segmentos_rutas[segmento_calle].append(ruta)

In [27]:
#Prueba:
dicc_segmentos_rutas['1403900010341300007']

[152]

## Data processing 02 - Asignar numero de rutas a los segmentos de ejesviales

In [28]:
# Crear un gdf de ejes viales con una columna lista para recibir el conteo de rutas (rutas_count)
ejesviales_rutas = ejesviales.copy()
ejesviales_rutas['rutas_count'] = 0

In [29]:
ejesviales_rutas.head(1)

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVEVIAL,CVESEG,NOMVIAL,TIPOVIAL,SENTIDO,TIPOSEN,AMBITO,CVEGEOSEG,rutas_transporte,geometry,rutas_count
0,140390001,14,039,0001,03975,00007,Patria,Avenida,Dos sentidos,2,Urbana,1403900010397500007,NaN,"MULTILINESTRING ((668243.519 2290955.957, 6682...",0


In [30]:
# Iterar en el diccionario creado para asignar el numero de rutas que pasan por cada segmento
for segmento in dicc_segmentos_rutas:
    idx = ejesviales_rutas['CVEGEOSEG'] == segmento
    ejesviales_rutas.loc[idx,'rutas_count'] = len(dicc_segmentos_rutas[segmento])

In [31]:
# Revisión rápida: En la dataframe de López Mateos el mayor número de rutas en una calle es 34, en López Mateos Sur.
ejesviales_rutas = ejesviales_rutas.sort_values(by = 'rutas_count', ascending = False)
ejesviales_rutas.head(2)

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVEVIAL,CVESEG,NOMVIAL,TIPOVIAL,SENTIDO,TIPOSEN,AMBITO,CVEGEOSEG,rutas_transporte,geometry,rutas_count
14446,141200001,14,120,0001,03541,00003,López Mateos Sur,Avenida,Dos sentidos,2,Urbana,1412000010354100003,NaN,"MULTILINESTRING ((666046.183 2283779.958, 6660...",35
14253,141200001,14,120,0001,00289,00005,López Mateos Sur,Avenida,Dos sentidos,2,Urbana,1412000010028900005,NaN,"MULTILINESTRING ((663767.488 2280042.525, 6638...",34


## Save to db

In [32]:
ejesviales_rutas = ejesviales_rutas.to_crs("EPSG:4326")
aup.gdf_to_db_slow(ejesviales_rutas,"rutastransporte_bystreet", 'segundopiso', if_exists='replace')